<a href="https://colab.research.google.com/github/gcosma/DECODEclinicalTrialCalc/blob/main/HospitalisationCalculator20Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hospitalisation clinical calculator - version 20 Feb 2025

Summary of overall admissions Admission rates (per 1000 persons per year) by type are shown in Figure 29. The overall rate for adults with ID was 351.6 per 1000 persons per year, compared with 246.4 per 1000 persons per year for controls. This difference was essentially due to the higher rate among emergency admissions (182.2 vs. 67.7 per 1000 persons per year), as elective rates were similar between groups." p70, Carey et al (2017)"


Two-sample Poisson Ratio Tests (Equal Sizes)

In [47]:
# @title Default title text
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from ipywidgets import interact, widgets
import math

def calculate_achieved_power(N, lambda1, lambda2, sig_level=0.05, alternative='two-sided'):
    """Calculate achieved power for given sample size and parameters"""
    rate_ratio = lambda1/lambda2

    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    ncp = np.log(rate_ratio) / np.sqrt(1/(lambda1*N) + 1/(lambda2*N))

    if alternative == 'two-sided':
        power = (1 - stats.norm.cdf(z_alpha - ncp) +
                stats.norm.cdf(-z_alpha - ncp))
    else:
        power = 1 - stats.norm.cdf(z_alpha - ncp)

    return power

def calculate_required_N(lambda1, lambda2, sig_level=0.05, power=0.80, alternative='two-sided', attrition_rate=0.0):
    """Calculate required sample size accounting for attrition"""
    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    z_beta = stats.norm.ppf(power)

    # Calculate base N (before attrition)
    N_base = (((z_alpha + z_beta)**2) * (lambda1 + lambda2)) / ((lambda1 - lambda2)**2)

    # Calculate N with attrition
    N_with_attrition = N_base / (1 - attrition_rate)

    return N_base, N_with_attrition

def get_recruitment_recommendation(required_N):
    """Get rounded up recruitment numbers that account for even distribution"""
    per_arm = math.ceil(required_N)
    total = per_arm * 2
    return per_arm, total

def display_clinical_power_analysis(example_N, lambda1, lambda2, target_power=0.90, sig_level=0.05, attrition_rate=0.10):
    """Display clinically oriented power analysis results with statistical documentation"""
    # Calculate base and attrition-adjusted N
    base_N, required_N = calculate_required_N(lambda1, lambda2, sig_level, target_power, attrition_rate=attrition_rate)

    # Calculate effective N and achieved power
    effective_N = example_N * (1 - attrition_rate)
    achieved_power = calculate_achieved_power(effective_N, lambda1, lambda2, sig_level)

    # Get recruitment recommendations for both base and attrition-adjusted
    base_per_arm, base_total = get_recruitment_recommendation(base_N)
    recruit_per_arm, total_recruitment = get_recruitment_recommendation(required_N)

    # Calculate expected events and other metrics
    rate_ratio = lambda1/lambda2
    relative_risk_reduction = ((lambda1 - lambda2)/lambda2) * 100
    power_achieved = achieved_power >= target_power

    html_output = f"""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #2c3e50; padding-bottom: 10px;'>
            Power Analysis Report for Poisson Rate Comparison
        </h2>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid {"#27ae60" if power_achieved else "#c0392b"}'>
            <h3 style='color: {"#27ae60" if power_achieved else "#c0392b"}; margin-top: 0;'>POWER STATUS: {'ACHIEVED' if power_achieved else 'NOT ACHIEVED'}</h3>
            <p style='font-size: 16px;'>
                Target Power: {target_power:.1%}<br>
                Achieved Power: {achieved_power:.1%}<br>
                {'✓ Sufficient sample size' if power_achieved else f'✗ Need {math.ceil(required_N - example_N)} more participants per arm'}
            </p>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #2980b9;'>
            <h3 style='color: #2980b9; margin-top: 0;'>SAMPLE SIZE REQUIREMENTS</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Before Attrition</h4>
                    <p style='font-size: 16px; font-weight: bold;'>
                        ➤ Per arm: {base_per_arm} participants<br>
                        ➤ Total: {base_total} participants
                    </p>
                </div>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>After {attrition_rate:.0%} Attrition</h4>
                    <p style='font-size: 16px; font-weight: bold;'>
                        ➤ Per arm: {recruit_per_arm} participants<br>
                        ➤ Total: {total_recruitment} participants
                    </p>
                </div>
            </div>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Current Sample Status</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Current Sample</h4>
                    <p style='font-size: 16px;'>
                        ➤ Per arm: {example_N:.0f} participants<br>
                        ➤ Total: {example_N * 2:.0f} participants
                    </p>
                </div>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>After {attrition_rate:.0%} Attrition</h4>
                    <p style='font-size: 16px;'>
                        ➤ Per arm: {effective_N:.0f} participants<br>
                        ➤ Total: {effective_N * 2:.0f} participants
                    </p>
                </div>
            </div>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Statistical Design Specifications</h3>
            <ul style='list-style-type: none; padding-left: 0;'>
                <li style='margin: 5px 0;'><strong>Study Design:</strong> Two-arm parallel group comparison of Poisson rates</li>
                <li style='margin: 5px 0;'><strong>Hypothesis Test:</strong> Two-sided test of rate equality</li>
                <li style='margin: 5px 0;'><strong>Type I Error (α):</strong> {sig_level:.2f}</li>
                <li style='margin: 5px 0;'><strong>Type II Error (β):</strong> {1-target_power:.2f}</li>
                <li style='margin: 5px 0;'><strong>Target Power (1-β):</strong> {target_power:.1%}</li>
            </ul>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Effect Size Parameters</h3>
            <ul style='list-style-type: none; padding-left: 0;'>
                <li style='margin: 5px 0;'><strong>Rate 1 (λ₁):</strong> {lambda1:.4f} events per person-year</li>
                <li style='margin: 5px 0;'><strong>Rate 2 (λ₂):</strong> {lambda2:.4f} events per person-year</li>
                <li style='margin: 5px 0;'><strong>Rate Ratio (λ₁/λ₂):</strong> {rate_ratio:.2f}</li>
                <li style='margin: 5px 0;'><strong>Relative Risk Reduction:</strong> {relative_risk_reduction:.1f}%</li>
            </ul>
        </div>
    </div>
    """
    display(HTML(html_output))

# Create interactive widget
@interact(
    example_N=widgets.FloatText(
        value=162,
        description='Current N per arm:',
        style={'description_width': '120px'}
    ),
    attrition_rate=widgets.FloatSlider(
        value=0.10,
        min=0.0,
        max=0.50,
        step=0.01,
        description='Attrition Rate:',
        style={'description_width': '120px'}
    ),
    target_power=widgets.FloatSlider(
        value=0.80,
        min=0.70,
        max=0.99,
        step=0.01,
        description='Target Power:',
        style={'description_width': '120px'}
    )
)
def run_clinical_analysis(example_N, attrition_rate, target_power):
    display_clinical_power_analysis(
        example_N=example_N,
        lambda1=0.1822,
        lambda2=0.0677,
        target_power=target_power,
        sig_level=0.05,
        attrition_rate=attrition_rate
    )

interactive(children=(FloatText(value=162.0, description='Current N per arm:', style=DescriptionStyle(descript…

In [44]:
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from ipywidgets import interact, widgets
import math

# [Previous statistical functions remain the same]
def calculate_achieved_power(N, lambda1, lambda2, sig_level=0.05, alternative='two-sided'):
    """Calculate achieved power for given sample size and parameters"""
    rate_ratio = lambda1/lambda2

    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    ncp = np.log(rate_ratio) / np.sqrt(1/(lambda1*N) + 1/(lambda2*N))

    if alternative == 'two-sided':
        power = (1 - stats.norm.cdf(z_alpha - ncp) +
                stats.norm.cdf(-z_alpha - ncp))
    else:
        power = 1 - stats.norm.cdf(z_alpha - ncp)

    return power

def calculate_required_N(lambda1, lambda2, sig_level=0.05, power=0.90, alternative='two-sided', attrition_rate=0.0):
    """Calculate required sample size accounting for attrition"""
    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    z_beta = stats.norm.ppf(power)
    N_base = (((z_alpha + z_beta)**2) * (lambda1 + lambda2)) / ((lambda1 - lambda2)**2)
    N_with_attrition = N_base / (1 - attrition_rate)
    return N_base, N_with_attrition

def get_recruitment_recommendation(required_N):
    per_arm = math.ceil(required_N)
    total = per_arm * 2
    return per_arm, total

def get_clinical_interpretation(lambda1, lambda2, achieved_power, target_power, example_N, required_N):
    """Provide clinically relevant interpretation of the results"""
    rate_ratio = lambda1/lambda2
    events_per_100py_1 = lambda1 * 100
    events_per_100py_2 = lambda2 * 100
    absolute_diff = (lambda1 - lambda2) * 100

    interpretation = f"""
    <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #34495e;'>
        <h3 style='color: #34495e; margin-top: 0;'>Clinical Interpretation</h3>

        <h4 style='color: #2c3e50;'>Expected Event Rates:</h4>
        <p>
        • Treatment group: {events_per_100py_1:.1f} events per 100 person-years<br>
        • Control group: {events_per_100py_2:.1f} events per 100 person-years<br>
        • Absolute difference: {absolute_diff:.1f} fewer events per 100 person-years
        </p>

        <h4 style='color: #2c3e50;'>Clinical Impact:</h4>
        <p>
        In a clinical setting, this means for every 100 patients followed for one year:<br>
        • Treatment group: Expect approximately {events_per_100py_1:.0f} events<br>
        • Control group: Expect approximately {events_per_100py_2:.0f} events<br>
        • Potential prevention of {absolute_diff:.0f} events per 100 patient-years
        </p>

        <h4 style='color: #2c3e50;'>Study Power Assessment:</h4>
        <p>"""

    if achieved_power >= target_power:
        interpretation += f"""
        Your current sample size is <span style='color: #27ae60;'>sufficient</span> to detect this difference with {achieved_power:.1%} power.<br>
        This means you have a {achieved_power:.1%} chance of detecting this treatment effect if it truly exists.
        </p>"""
    else:
        more_needed = math.ceil(required_N - example_N)
        interpretation += f"""
        Your study is currently <span style='color: #c0392b;'>underpowered</span> at {achieved_power:.1%}.<br>
        • To reach {target_power:.1%} power, you need {more_needed} more participants per arm<br>
        • Without additional recruitment, you have a {achieved_power:.1%} chance of detecting the treatment effect<br>
        • This increases your risk of missing a true treatment effect
        </p>"""

    return interpretation

def display_clinical_power_analysis(example_N, lambda1, lambda2, target_power=0.90, sig_level=0.05, attrition_rate=0.10):
    """Display clinically oriented power analysis results with enhanced interpretation"""
    # Calculate key metrics
    base_N, required_N = calculate_required_N(lambda1, lambda2, sig_level, target_power, attrition_rate=attrition_rate)
    effective_N = example_N * (1 - attrition_rate)
    achieved_power = calculate_achieved_power(effective_N, lambda1, lambda2, sig_level)
    base_per_arm, base_total = get_recruitment_recommendation(base_N)
    recruit_per_arm, total_recruitment = get_recruitment_recommendation(required_N)

    # Get clinical interpretation
    clinical_interpretation = get_clinical_interpretation(
        lambda1, lambda2, achieved_power, target_power, example_N, required_N
    )

    html_output = f"""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #2c3e50; padding-bottom: 10px;'>
            Clinical Trial Power Analysis Report
        </h2>

        {clinical_interpretation}

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #2980b9;'>
            <h3 style='color: #2980b9; margin-top: 0;'>Recruitment Needs</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Current Status</h4>
                    <p style='font-size: 16px;'>
                        • Currently enrolled: {example_N:.0f} per arm<br>
                        • After {attrition_rate:.0%} dropout: {effective_N:.0f} per arm<br>
                        • Total enrolled: {example_N * 2:.0f} participants
                    </p>
                </div>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Target Numbers</h4>
                    <p style='font-size: 16px;'>
                        • Need to enroll: {recruit_per_arm} per arm<br>
                        • Total needed: {total_recruitment} participants<br>
                        • Accounts for {attrition_rate:.0%} expected dropout
                    </p>
                </div>
            </div>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Study Design Summary</h3>
            <p>
            • Type of study: Two-arm parallel group comparison<br>
            • Primary analysis: Comparison of event rates between groups<br>
            • Statistical test: Two-sided test at {sig_level:.0%} significance level<br>
            • Target power: {target_power:.1%} chance of detecting the specified difference<br>
            • Current power: {achieved_power:.1%} with current sample size
            </p>
        </div>
    </div>
    """
    display(HTML(html_output))

# Interactive widget setup
@interact(
    example_N=widgets.FloatText(
        value=162,
        description='Current enrolled per arm:',
        style={'description_width': '150px'}
    ),
    attrition_rate=widgets.FloatSlider(
        value=0.10,
        min=0.0,
        max=0.50,
        step=0.01,
        description='Expected dropout rate:',
        style={'description_width': '150px'}
    ),
    target_power=widgets.FloatSlider(
        value=0.90,
        min=0.70,
        max=0.99,
        step=0.01,
        description='Desired power:',
        style={'description_width': '150px'}
    )
)
def run_clinical_analysis(example_N, attrition_rate, target_power):
    display_clinical_power_analysis(
        example_N=example_N,
        lambda1=0.1822,
        lambda2=0.0677,
        target_power=target_power,
        sig_level=0.05,
        attrition_rate=attrition_rate
    )

interactive(children=(FloatText(value=162.0, description='Current enrolled per arm:', style=DescriptionStyle(d…

In [43]:
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from ipywidgets import interact, widgets
import math

def calculate_achieved_power(N, lambda1, lambda2, sig_level=0.05, alternative='two-sided'):
    """Calculate achieved power for given sample size and parameters"""
    rate_ratio = lambda1/lambda2

    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    ncp = np.log(rate_ratio) / np.sqrt(1/(lambda1*N) + 1/(lambda2*N))

    if alternative == 'two-sided':
        power = (1 - stats.norm.cdf(z_alpha - ncp) +
                stats.norm.cdf(-z_alpha - ncp))
    else:
        power = 1 - stats.norm.cdf(z_alpha - ncp)

    return power

def calculate_required_N(lambda1, lambda2, sig_level=0.05, power=0.90, alternative='two-sided', attrition_rate=0.0):
    """Calculate required sample size accounting for attrition"""
    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level/2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    z_beta = stats.norm.ppf(power)

    # Calculate base N (before attrition)
    N_base = (((z_alpha + z_beta)**2) * (lambda1 + lambda2)) / ((lambda1 - lambda2)**2)

    # Calculate N with attrition
    N_with_attrition = N_base / (1 - attrition_rate)

    return N_base, N_with_attrition

def get_recruitment_recommendation(required_N):
    """Get rounded up recruitment numbers that account for even distribution"""
    per_arm = math.ceil(required_N)
    total = per_arm * 2
    return per_arm, total

def display_clinical_power_analysis(example_N, lambda1, lambda2, target_power=0.90, sig_level=0.05, attrition_rate=0.10):
    """Display clinically oriented power analysis results with statistical documentation"""
    # Calculate base and attrition-adjusted N
    base_N, required_N = calculate_required_N(lambda1, lambda2, sig_level, target_power, attrition_rate=attrition_rate)

    # Calculate effective N and achieved power
    effective_N = example_N * (1 - attrition_rate)
    achieved_power = calculate_achieved_power(effective_N, lambda1, lambda2, sig_level)

    # Get recruitment recommendations for both base and attrition-adjusted
    base_per_arm, base_total = get_recruitment_recommendation(base_N)
    recruit_per_arm, total_recruitment = get_recruitment_recommendation(required_N)

    # Calculate expected events and other metrics
    rate_ratio = lambda1/lambda2
    relative_risk_reduction = ((lambda1 - lambda2)/lambda2) * 100
    power_achieved = achieved_power >= target_power

    html_output = f"""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #2c3e50; padding-bottom: 10px;'>
            Power Analysis Report for Poisson Rate Comparison
        </h2>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid {"#27ae60" if power_achieved else "#c0392b"}'>
            <h3 style='color: {"#27ae60" if power_achieved else "#c0392b"}; margin-top: 0;'>POWER STATUS: {'ACHIEVED' if power_achieved else 'NOT ACHIEVED'}</h3>
            <p style='font-size: 16px;'>
                Target Power: {target_power:.1%}<br>
                Achieved Power: {achieved_power:.1%}<br>
                {'✓ Sufficient sample size' if power_achieved else f'✗ Need {math.ceil(required_N - example_N)} more participants per arm'}
            </p>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #2980b9;'>
            <h3 style='color: #2980b9; margin-top: 0;'>SAMPLE SIZE REQUIREMENTS</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Before Attrition</h4>
                    <p style='font-size: 16px; font-weight: bold;'>
                        ➤ Per arm: {base_per_arm} participants<br>
                        ➤ Total: {base_total} participants
                    </p>
                </div>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>After {attrition_rate:.0%} Attrition</h4>
                    <p style='font-size: 16px; font-weight: bold;'>
                        ➤ Per arm: {recruit_per_arm} participants<br>
                        ➤ Total: {total_recruitment} participants
                    </p>
                </div>
            </div>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Current Sample Status</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>Current Sample</h4>
                    <p style='font-size: 16px;'>
                        ➤ Per arm: {example_N:.0f} participants<br>
                        ➤ Total: {example_N * 2:.0f} participants
                    </p>
                </div>
                <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                    <h4 style='color: #2c3e50; margin-top: 0;'>After {attrition_rate:.0%} Attrition</h4>
                    <p style='font-size: 16px;'>
                        ➤ Per arm: {effective_N:.0f} participants<br>
                        ➤ Total: {effective_N * 2:.0f} participants
                    </p>
                </div>
            </div>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Statistical Design Specifications</h3>
            <ul style='list-style-type: none; padding-left: 0;'>
                <li style='margin: 5px 0;'><strong>Study Design:</strong> Two-arm parallel group comparison of Poisson rates</li>
                <li style='margin: 5px 0;'><strong>Hypothesis Test:</strong> Two-sided test of rate equality</li>
                <li style='margin: 5px 0;'><strong>Type I Error (α):</strong> {sig_level:.2f}</li>
                <li style='margin: 5px 0;'><strong>Type II Error (β):</strong> {1-target_power:.2f}</li>
                <li style='margin: 5px 0;'><strong>Target Power (1-β):</strong> {target_power:.1%}</li>
            </ul>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 20px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Effect Size Parameters</h3>
            <ul style='list-style-type: none; padding-left: 0;'>
                <li style='margin: 5px 0;'><strong>Rate 1 (λ₁):</strong> {lambda1:.4f} events per person-year</li>
                <li style='margin: 5px 0;'><strong>Rate 2 (λ₂):</strong> {lambda2:.4f} events per person-year</li>
                <li style='margin: 5px 0;'><strong>Rate Ratio (λ₁/λ₂):</strong> {rate_ratio:.2f}</li>
                <li style='margin: 5px 0;'><strong>Relative Risk Reduction:</strong> {relative_risk_reduction:.1f}%</li>
            </ul>
        </div>
    </div>
    """
    display(HTML(html_output))

# Create interactive widget
@interact(
    example_N=widgets.FloatText(
        value=162,
        description='Current N per arm:',
        style={'description_width': '120px'}
    ),
    attrition_rate=widgets.FloatSlider(
        value=0.10,
        min=0.0,
        max=0.50,
        step=0.01,
        description='Attrition Rate:',
        style={'description_width': '120px'}
    ),
    target_power=widgets.FloatSlider(
        value=0.90,
        min=0.70,
        max=0.99,
        step=0.01,
        description='Target Power:',
        style={'description_width': '120px'}
    )
)
def run_clinical_analysis(example_N, attrition_rate, target_power):
    display_clinical_power_analysis(
        example_N=example_N,
        lambda1=0.1822,
        lambda2=0.0677,
        target_power=target_power,
        sig_level=0.05,
        attrition_rate=attrition_rate
    )

interactive(children=(FloatText(value=162.0, description='Current N per arm:', style=DescriptionStyle(descript…